In [237]:
import numpy as np
import itertools
import random
import math
import itertools as it
from itertools import product, combinations

Написать функцию формирования порождающей матрицы кода Рида
Маллера (r,m) в каноническом виде для произвольно заданных r и m.

In [238]:
# Формируем матрицу размерности m столбцов
def basis_matrix(columns):
    return list(product([0, 1], repeat=columns))

# создаем вектор на основе индексов и количества столбцов
def create_vector(index_set, total_columns):
    if len(index_set) == 0:
        return np.ones(2 ** total_columns, dtype=int)

    vec = []
    basis = basis_matrix(total_columns)
    for pattern in basis:
        result = 1
        for idx in index_set:
            result *= (pattern[idx] + 1) % 2
        vec.append(result)

    return vec

# генерируем комбинации
def generate_combinations(total, limit):
    indices = np.arange(total)
    result_sets = []
    for size in range(len(indices) + 1):
        for combo in combinations(indices, size):
            if len(combo) <= limit:
                result_sets.append(combo)

    return result_sets



In [239]:

def matrix_R_M(row_limit, total_cols):
    size = 0
    for i in range(r + 1):
        size += math.comb(m, i)
    matrix = np.zeros((size, pow(2, m)), dtype=int)
    index = 0
    for i in generate_combinations(m, r):
        matrix[index] = create_vector(i, m)
        index += 1
    return matrix


r,m = 2,4
G = matrix_R_M(r,m)
print(G)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]]


Реализовать алгоритм мажоритарного декодирования для кода Рида
Маллера

In [240]:
#генерируем и сортируем все комбинаци r элементов из m
def sort_for_major(m, r):
    iterable = np.arange(m, dtype=int)
    temp = list(combinations(iterable, r))
    if temp:
        temp.sort()
    result = np.zeros((math.comb(m, r), r), dtype=int)
    for i in range(len(temp)):
        result[i] = temp[i]
    return result

#дополнение множества
def compl(I, m):
    return [i for i in range(m) if i not in I]

#нахождение строк базиса
def get_H_I(I, m):

    H_I = []
    for words in basis_matrix(m):
        f = 1
        for j in I:
            f *= (words[j] + 1) % 2
        if f == 1:
            H_I.append(words)
    return H_I

#создает вектор на основе заданногомножества индексов и значений
def create_vector_t(I, m, t):
    if len(I) == 0:
        return np.ones(2 ** m, int)
    else:
        v = []
        for words in basis_matrix(m):
            f = 1
            for j in I:
                f *= (words[j] + t[j] + 1) % 2
            v.append(f)
        return v




In [241]:
def major(w, r, m, size):
    i = r
    w_r = w.copy()
    Mi = np.zeros(size, dtype=int)
    max_weight = 2 ** (m - r - 1) - 1
    index = 0

    while True:
        combination_found = False

        for J in sort_for_major(m, i):
            zero_count, one_count = 0, 0

            for t in get_H_I(J, m):
                komp = compl(J, m)
                V = create_vector_t(komp, m, t)
                c = np.dot(w_r, V) % 2

                if c == 0:
                    zero_count += 1
                elif c == 1:
                    one_count += 1

            if zero_count > max_weight and one_count > max_weight:
                return

            if zero_count > pow(2, m - i - 1):
                Mi[index] = 0
                index += 1
                combination_found = True

            if one_count > pow(2, m - i - 1):
                Mi[index] = 1
                index += 1
                w_r = (w_r + create_vector(J, m)) % 2
                combination_found = True

        if not combination_found:
            break

        if i > 0:
            if len(w_r) < max_weight:
                Mi[index:index + len(sort_for_major(m, r + 1))] = 0
                index += len(sort_for_major(m, r + 1))
                break
            i -= 1
        else:
            break
    reversed(Mi)
    return Mi



In [242]:
def generate_word(u, G, error_count):
    v = np.dot(u,G)%2
    print ("u: ", u)
    print("v: ", v)

    #Внести ошибку в сформированное слово
    i_f = np.random.choice(len(u), size=error_count, replace=False)
    for index in (i_f):
        v[index] = (v[index]+1)%2
    print("Полученное кодовое слово с ошибкой: ", v)


    return v

Провести экспериментальную проверку алгоритма декодирования для
кода Рида-Маллера (2,4)

In [243]:
def check(u,g,count_mistake):

  match count_mistake:
    case 1:
      Err = generate_word(u,G, 1)
      print("Слово с однократной ошибкой:", Err)
    case 2:
      Err = generate_word(u,G, 2)
      print("Слово с двукратной ошибкой:", Err)
  Decoded_word = major(Err, 2, 4, len(G))
  if Decoded_word is None:
      print("Необходима повторная отправка сообщения\n")
  else:
      print("Исправленное слово:", Decoded_word)
      V2 = Decoded_word.dot(G) % 2
      print("Проверка:", V2)

In [244]:
# Проверка однократной ошибки
u = np.array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
check(u,G,1)

u:  [1 0 0 0 1 1 0 0 0 1 1]
v:  [1 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1]
Полученное кодовое слово с ошибкой:  [1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1]
Слово с однократной ошибкой: [1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1]
Исправленное слово: [1 0 0 0 1 1 0 0 0 1 1]
Проверка: [1 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1]


In [245]:
# Проверка двукратной ошибки
u = np.array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
check(u,G,2)

u:  [1 0 0 0 1 1 0 0 0 1 1]
v:  [1 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1]
Полученное кодовое слово с ошибкой:  [1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1]
Слово с двукратной ошибкой: [1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1]
Необходима повторная отправка сообщения

